# IceCube Detector SNDAQ Formats

Try to consolidate DOM geometry and efficiency data from various files into one database in some format, e.g., JSON or FITS.

In [1]:
import numpy as np
from astropy.io import ascii, fits
from astropy.table import Table
import xml.etree.ElementTree as ET
import json

## Load DOM Information

Use the existing text and XML files to load the detector data structures.

### IceCube Geometry

In [2]:
geometry = ascii.read('../../data/config/Icecube_geometry.20110414.complete.txt',
                      names=['string', 'position', 'x', 'y', 'z'])

## JSON Output

In [3]:
detdata = []
detector = {}
detector['IceCube'] = []

detxml = ET.parse('../../data/config/detector.xml')
detroot = detxml.getroot()
for dom in detroot.iter('dom'):
    id_   = int(dom.get('id'), 16)
    str_  = int(dom.get('string'))
    pos_  = int(dom.get('position'))
    name_ = dom.get('name')
    eff_  = float(dom.get('efficiency'))
    dt_   = float(dom.get('deadtime'))
    
    cut = np.logical_and(geometry['string'] == str_, geometry['position'] == pos_)
    x_, y_, z_ = [float(geometry[cut][u]) for u in ['x', 'y', 'z']]
    
    subdet_, type_ = None, None
    if str_ <= 80:
        subdet_ = 'I3'
        type_   = 'DOM'
    elif str_ > 80 and str_ <= 86:
        subdet_ = 'DC'
        type_   = 'DOM_HQE'
    else:
        raise SystemExit('Unknown string ID {:d}'.format(str_))
        
    detdata.append((name_, id_, type_, subdet_, str_, pos_, x_, y_, z_, eff_, dt_))
    
    detector['IceCube'].append({
            'name'       : name_,
            'om_type'    : type_,
            'str_type'   : subdet_,
            'id'         : id_,
            'string'     : str_,
            'position'   : pos_,
            'x'          : { 'value' : x_, 'unit' : 'm' },
            'y'          : { 'value' : y_, 'unit' : 'm' },
            'z'          : { 'value' : z_, 'unit' : 'm' },
            'efficiency' : eff_,
            'deadtime'   : { 'value' : dt_, 'unit' : 'us' }
        })

In [10]:
with open('detector_icecube86.json', 'w') as f:
#     json.dump(detector, f)
    f.write(json.dumps(detector, indent=4, sort_keys=False))

In [11]:
with open('detector_icecube86.json', 'r') as f:
    data = json.load(f)

In [5]:
len(data['IceCube'])

5160

In [6]:
data['IceCube']

[{'deadtime': {'unit': 'us', 'value': 250.0},
  'efficiency': 1.0,
  'id': 47303335284587,
  'name': 'Mouse_Trap',
  'om_type': 'DOM',
  'position': 1,
  'str_type': 'I3',
  'string': 1,
  'x': {'unit': 'm', 'value': -256.14},
  'y': {'unit': 'm', 'value': -521.08},
  'z': {'unit': 'm', 'value': 496.03}},
 {'deadtime': {'unit': 'us', 'value': 250.0},
  'efficiency': 1.0,
  'id': 20579555797555,
  'name': 'Trouble',
  'om_type': 'DOM',
  'position': 2,
  'str_type': 'I3',
  'string': 1,
  'x': {'unit': 'm', 'value': -256.14},
  'y': {'unit': 'm', 'value': -521.08},
  'z': {'unit': 'm', 'value': 479.01}},
 {'deadtime': {'unit': 'us', 'value': 250.0},
  'efficiency': 1.0,
  'id': 106634453247646,
  'name': 'Monopoly',
  'om_type': 'DOM',
  'position': 3,
  'str_type': 'I3',
  'string': 1,
  'x': {'unit': 'm', 'value': -256.14},
  'y': {'unit': 'm', 'value': -521.08},
  'z': {'unit': 'm', 'value': 461.99}},
 {'deadtime': {'unit': 'us', 'value': 250.0},
  'efficiency': 1.0,
  'id': 22580079

## FITS Output using `astropy.table`

In [7]:
t = Table(rows=detdata, names=('name', 'id', 'om_type', 'str_type',
                               'string', 'position', 'x', 'y', 'z',
                               'efficiency', 'deadtime'),
          meta={'name': 'IceCube Detector'},
          dtype=('S50', 'i8', 'S8', 'S10',
                 'i4', 'i4', 'f8', 'f8', 'f8',
                 'f8', 'f8'))

t['x'].unit = 'm'
t['y'].unit = 'm'
t['z'].unit = 'm'
t['deadtime'].unit = 'microsecond'
t.write('detector_icecube86.fits', format='fits', overwrite=True)

In [8]:
read_tab = Table.read('detector_icecube86.fits')

In [9]:
read_tab

name,id,om_type,str_type,string,position,x,y,z,efficiency,deadtime
,,,,,,m,m,m,,us
bytes50,int64,bytes8,bytes10,int32,int32,float64,float64,float64,float64,float64
Mouse_Trap,47303335284587,DOM,I3,1,1,-256.14,-521.08,496.03,1.0,250.0
Trouble,20579555797555,DOM,I3,1,2,-256.14,-521.08,479.01,1.0,250.0
Monopoly,106634453247646,DOM,I3,1,3,-256.14,-521.08,461.99,1.0,250.0
Eftersteka,225800795471873,DOM,I3,1,4,-256.14,-521.08,444.97,1.0,250.0
Stuva,20512416651416,DOM,I3,1,5,-256.14,-521.08,427.95,1.0,250.0
Chutes_And_Ladders,257280767891421,DOM,I3,1,6,-256.14,-521.08,410.93,1.0,250.0
Berberis_Thungbergii,8829211658416,DOM,I3,1,7,-256.14,-521.08,393.91,1.0,250.0
Risk,150606365672579,DOM,I3,1,8,-256.14,-521.08,376.88,1.0,250.0
